In [1]:
#####general imports
import os
import pandas as pd
import numpy as np

import itertools as it
from random import randint
import statistics as stat

#####sample extraction
import glob

#####plotting
import plotly.express as px
import plotly.graph_objects as go

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from upsetplot import from_indicators
from upsetplot import UpSet
from upsetplot import plot

#####clustering
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, leaves_list, fcluster
from scipy.spatial.distance import pdist 

#####PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#####stats
from sklearn.linear_model import LinearRegression
from sklearn.covariance import EllipticEnvelope
from scipy.stats import norm

/Users/tajo5912/jhub_venv/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
species='human'

df = pd.read_csv('/Users/tajo5912/cistrome/'+species+'_factor_full_QC.txt', sep='\t')
print(len(df))

11348


In [28]:
# ###these are their definitions of passing
# ###mapping
# df[df['FastQC'] > 25]                  #sequence quality
# df[df['UniquelyMappedRatio'] > 0.6]    #unique sequences
# df[df['PBC'] > 0.8]                    #when the data is subsampled down, how many unique regions are there

# ###peak calling
# df[df['PeaksFoldChangeAbove10'] > 500] #how many peaks called by macs
# df[df['FRiP'] > 0.01]                  #evaluating signal to noise ratio
# df[df['PeaksUnionDHSRatio'] > 0.7]     #overlap with accessible regions

In [3]:
df['mapping_score'] = ((df['FastQC'] > 25).astype(int) + 
                       (df['UniquelyMappedRatio'] > 0.6).astype(int) + 
                       (df['PBC'] > 0.8).astype(int))
df['peak_score'] = ((df['PeaksFoldChangeAbove10'] > 500).astype(int) +
                    (df['FRiP'] > 0.01).astype(int) +
                    (df['PeaksUnionDHSRatio'] > 0.7).astype(int))
df['overall_score'] = df['mapping_score'] + df['peak_score']

In [4]:
df=df[df['overall_score'] != 0]
print(len(df))
df = df[df['mapping_score'] != 0]
df = df[df['peak_score'] != 0]
df

11341


,DCid,Species,GSMID,Factor,Cell_line,Cell_type,Tissue_type,FastQC,UniquelyMappedRatio,PBC,PeaksFoldChangeAbove10,FRiP,PeaksUnionDHSRatio,mapping_score,peak_score,overall_score
0,1,Homo sapiens,GSM448026,BTAF1,HeLa,Epithelium,Cervix,9.0,0.0502,0.880,406.0,0.094809,0.352373,1,1,2
1,2,Homo sapiens,GSM448027,GAPDH,HeLa,Epithelium,Cervix,9.0,0.0274,0.931,248.0,0.175196,0.247082,1,1,2
2,4,Homo sapiens,GSM540707,EGR1,K562,Erythroblast,Bone Marrow,9.0,0.0845,0.893,500.0,0.036462,0.522970,1,1,2
3,6,Homo sapiens,GSM460127,TCF4,LS174T,Epithelium,Colon,9.0,0.0007,0.819,7.0,0.021364,0.333333,1,1,2
10,84,Homo sapiens,GSM353640,AR,22RV1,Epithelium,Prostate,32.0,0.4240,0.848,23.0,0.000838,0.795918,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11343,88969,Homo sapiens,GSM1706704,HDAC2,1184,Stem cell,None,36.0,0.5314,0.847,18.0,0.001077,0.857143,2,1,3
11344,88976,Homo sapiens,GSM2719738,ARID1A,HCT-116,None,HCT116,36.0,0.8870,0.994,31.0,0.106809,0.877200,3,2,5
11345,88980,Homo sapiens,GSM2719736,ARID1A,HCT-116,None,HCT116,36.0,0.8826,0.998,2.0,0.000073,1.000000,3,1,4
11346,88981,Homo sapiens,GSM2719737,ARID1A,HCT-116,None,HCT116,36.0,0.8854,0.994,14.0,0.054080,0.877600,3,2,5


In [21]:
if species == 'human':
    df['Cell_line']=df['Cell_line'].str.replace(' ', '_')
    df['Cell_line']=df['Cell_line'].str.replace('-', '_')
    df['Cell_line']=df['Cell_line'].str.replace('/', '_')
    df['Cell_line']=df['Cell_line'].str.replace('+', 'plus')
    df['Cell_line']=df['Cell_line'].str.replace(';', '_')
    
if species == 'mouse':
    df['Cell_type']=df['Cell_type'].str.replace(' ', '_')
    df['Cell_type']=df['Cell_type'].str.replace('-', '_')
    df['Cell_type']=df['Cell_type'].str.replace('/', '_')
    df['Cell_type']=df['Cell_type'].str.replace('+', 'plus')
    df['Cell_type']=df['Cell_type'].str.replace(';', '_')
    df['Cell_line'] = df['Cell_type']

In [22]:
df['samp_id'] = (df['DCid'].astype(str) + '_' +df['Factor'] + '_' + df['Cell_line'] + '_' + 
                 (df['mapping_score']).astype(str) + '_' +(df['peak_score']).astype(str))
df=df.reset_index(drop=True)

In [23]:
df

,DCid,Species,GSMID,Factor,Cell_line,Cell_type,Tissue_type,FastQC,UniquelyMappedRatio,PBC,PeaksFoldChangeAbove10,FRiP,PeaksUnionDHSRatio,mapping_score,peak_score,overall_score,samp_id
0,1,Homo sapiens,GSM448026,BTAF1,HeLa,Epithelium,Cervix,9.0,0.0502,0.880,406.0,0.094809,0.352373,1,1,2,1_BTAF1_HeLa_1_1
1,2,Homo sapiens,GSM448027,GAPDH,HeLa,Epithelium,Cervix,9.0,0.0274,0.931,248.0,0.175196,0.247082,1,1,2,2_GAPDH_HeLa_1_1
2,4,Homo sapiens,GSM540707,EGR1,K562,Erythroblast,Bone Marrow,9.0,0.0845,0.893,500.0,0.036462,0.522970,1,1,2,4_EGR1_K562_1_1
3,6,Homo sapiens,GSM460127,TCF4,LS174T,Epithelium,Colon,9.0,0.0007,0.819,7.0,0.021364,0.333333,1,1,2,6_TCF4_LS174T_1_1
4,84,Homo sapiens,GSM353640,AR,22RV1,Epithelium,Prostate,32.0,0.4240,0.848,23.0,0.000838,0.795918,2,1,3,84_AR_22RV1_2_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9506,88969,Homo sapiens,GSM1706704,HDAC2,1184,Stem cell,None,36.0,0.5314,0.847,18.0,0.001077,0.857143,2,1,3,88969_HDAC2_1184_2_1
9507,88976,Homo sapiens,GSM2719738,ARID1A,HCT_116,None,HCT116,36.0,0.8870,0.994,31.0,0.106809,0.877200,3,2,5,88976_ARID1A_HCT_116_3_2
9508,88980,Homo sapiens,GSM2719736,ARID1A,HCT_116,None,HCT116,36.0,0.8826,0.998,2.0,0.000073,1.000000,3,1,4,88980_ARID1A_HCT_116_3_1
9509,88981,Homo sapiens,GSM2719737,ARID1A,HCT_116,None,HCT116,36.0,0.8854,0.994,14.0,0.054080,0.877600,3,2,5,88981_ARID1A_HCT_116_3_2


In [16]:
# glob.glob('/Users/tajo5912/cistrome/human_factor/*')
# _sort_peaks.narrowPeak.bed
for i in range(len(df)):
    os.system('rsync /Users/tajo5912/cistrome/'+species+'_factor/' + str(df['DCid'][i])+
          '_sort_peaks.narrowPeak.bed '+
          '/Users/tajo5912/cistrome/'+species+'_factor_relab/'+
              df['samp_id'][i]+
          '.bed')

In [24]:
df.to_csv('/Users/tajo5912/cistrome/'+species+'_factor_QC_filtered.txt',
         sep='\t', index=False)
df.to_csv('/scratch/Shares/dowell/tajo/cistrome_factors/'+species+'_factor_QC_filtered.txt',
         sep='\t', index=False)